In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pretty_midi
import librosa
import copy

In [ ]:
f0_path = '../../../../Google Drive/Dagstuhl Dataset/Annotations/DLI_QuartetB_Take4_PYIN/'
sop_path = f0_path + 'DLI_QuartetB_Take4_S1_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'
alt_path = f0_path + 'DLI_QuartetB_Take4_A2_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'
ten_path = f0_path + 'DLI_QuartetB_Take4_T2_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'
bas_path = f0_path + 'DLI_QuartetB_Take4_B2_LRX_vamp_pyin_pyin_smoothedpitchtrack.csv'

midi_path = '../../../../Google Drive/Dagstuhl Dataset/Annotations/SheetMusic_aligned_AudioLabsBeats/Bruckner_WAB023_BrianMarble-CPDL_warpedTo_DLI_QuartetB_Take4_Stereo_STM.mid'

In [ ]:
# load files
sop = pd.read_csv(sop_path, header=None).values
alt = pd.read_csv(alt_path, header=None).values
ten = pd.read_csv(ten_path, header=None).values
bas = pd.read_csv(bas_path, header=None).values

sop[sop[:, 1] < 0, 1] = 0
alt[alt[:, 1] < 0, 1] = 0
ten[ten[:, 1] < 0, 1] = 0
bas[bas[:, 1] < 0, 1] = 0

# get onset, offset, MIDI center frequency from aligned MIDI file
patt = pretty_midi.PrettyMIDI(midi_path)
sop_MIDI = np.empty((0, 3))
alt_MIDI = np.empty((0, 3))
ten_MIDI = np.empty((0, 3))
bas_MIDI = np.empty((0, 3))

for note in patt.instruments[0].notes:
    sop_MIDI = np.vstack((sop_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))
for note in patt.instruments[1].notes:
    alt_MIDI = np.vstack((alt_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))
for note in patt.instruments[2].notes:
    ten_MIDI = np.vstack((ten_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))
for note in patt.instruments[3].notes:
    bas_MIDI = np.vstack((bas_MIDI, (note.start, note.end, 440*2**((note.pitch-69)/12))))

In [ ]:
# visualize trajectories
%matplotlib notebook
plt.figure()
plt.plot(sop[:, 0], sop[:, 1], marker='.', linestyle='')
plt.plot(alt[:, 0], alt[:, 1], marker='.', linestyle='')
plt.plot(ten[:, 0], ten[:, 1], marker='.', linestyle='')
plt.plot(bas[:, 0], bas[:, 1], marker='.', linestyle='')
plt.ylim((0, 800))


In [ ]:
def midi_to_trajectory(MIDI_info, t_axis):
    traj = np.hstack((t_axis.reshape(-1, 1), np.zeros(len(t_axis)).reshape(-1,1)))
    
    for i in range(MIDI_info.shape[0]):
        # get indices of trajectory
        t_start_idx = np.argmin(np.abs(t_axis - MIDI_info[i, 0]))
        t_end_idx = np.argmin(np.abs(t_axis - MIDI_info[i, 1]))
        
        note_len_idx = t_end_idx - t_start_idx
        traj[t_start_idx:t_end_idx, 1] = [MIDI_info[i, 2]]*note_len_idx
    
    return traj

def compute_notewise_mean_median(MIDI_info, traj):
    nw_mean = np.hstack((MIDI_info[:, 0:2], np.zeros(MIDI_info.shape[0]).reshape(-1, 1)))
    nw_median = np.hstack((MIDI_info[:, 0:2], np.zeros(MIDI_info.shape[0]).reshape(-1, 1)))
    
    traj_nan = copy.deepcopy(traj)
    traj_nan[traj_nan[:, 1] == 0, 1] = np.nan
    
    for i in range(MIDI_info.shape[0]):
        # get indices of trajectory
        t_start_idx = np.argmin(np.abs(traj_nan[:, 0] - MIDI_info[i, 0]))
        t_end_idx = np.argmin(np.abs(traj_nan[:, 0] - MIDI_info[i, 1]))
        
        nw_mean[i, 2] = np.nanmean(traj_nan[t_start_idx:t_end_idx, 1])
        nw_median[i, 2] = np.nanmedian(traj_nan[t_start_idx:t_end_idx, 1])
        
    return nw_mean, nw_median

In [ ]:
# get mean of trajectories in note regions
traj_MIDI_sop = midi_to_trajectory(sop_MIDI, sop[:, 0])
traj_MIDI_alt = midi_to_trajectory(alt_MIDI, alt[:, 0])
traj_MIDI_ten = midi_to_trajectory(ten_MIDI, ten[:, 0])
traj_MIDI_bas = midi_to_trajectory(bas_MIDI, bas[:, 0])

nw_mean_sop, nw_median_sop = compute_notewise_mean_median(sop_MIDI, sop)
nw_mean_alt, nw_median_alt = compute_notewise_mean_median(alt_MIDI, alt)
nw_mean_ten, nw_median_ten = compute_notewise_mean_median(ten_MIDI, ten)
nw_mean_bas, nw_median_bas = compute_notewise_mean_median(bas_MIDI, bas)

traj_mean_sop = midi_to_trajectory(nw_mean_sop, sop[:, 0])
traj_mean_alt = midi_to_trajectory(nw_mean_alt, alt[:, 0])
traj_mean_ten = midi_to_trajectory(nw_mean_ten, ten[:, 0])
traj_mean_bas = midi_to_trajectory(nw_mean_bas, bas[:, 0])

traj_median_sop = midi_to_trajectory(nw_median_sop, sop[:, 0])
traj_median_alt = midi_to_trajectory(nw_median_alt, alt[:, 0])
traj_median_ten = midi_to_trajectory(nw_median_ten, ten[:, 0])
traj_median_bas = midi_to_trajectory(nw_median_bas, bas[:, 0])

In [ ]:
%matplotlib notebook
plt.figure()
plt.plot(traj_MIDI_sop[:, 0], traj_MIDI_sop[:, 1], color='k', marker='.', linestyle='')
plt.plot(traj_MIDI_alt[:, 0], traj_MIDI_alt[:, 1], color='k', marker='.', linestyle='')
plt.plot(traj_MIDI_ten[:, 0], traj_MIDI_ten[:, 1], color='k', marker='.', linestyle='')
plt.plot(traj_MIDI_bas[:, 0], traj_MIDI_bas[:, 1], color='k', marker='.', linestyle='')

plt.plot(traj_mean_sop[:, 0], traj_median_sop[:, 1], marker='.', linestyle='')
plt.plot(traj_mean_alt[:, 0], traj_median_alt[:, 1], marker='.', linestyle='')
plt.plot(traj_mean_ten[:, 0], traj_median_ten[:, 1], marker='.', linestyle='')
plt.plot(traj_mean_bas[:, 0], traj_median_bas[:, 1], marker='.', linestyle='')
#plt.ylim((0, 800))

In [ ]:
# compute deviation between score and mean f0
def compute_interval_in_cents(f_hz, f_ref_hz=55):
    """
    Computes interval in cents between two or multiple frequencies
    :param f_hz: frequency or array of frequencies given in Hz
    :param f_ref_hz: reference frequency or array of reference frequencies given in Hz
    :return: interval(s) in cents
    """
    f_new = copy.deepcopy(f_hz)
    f_ref_new = copy.deepcopy(f_ref_hz)

    if not np.isscalar(f_new):
        f_new[f_new == 0] = np.nan

    if not np.isscalar(f_ref_new):
        f_ref_new[f_ref_new == 0] = np.nan

    int_cents = 1200 * np.log2(np.divide(f_new, f_ref_new))

    return int_cents

dev_sop = compute_interval_in_cents(traj_median_sop[:, 1], traj_MIDI_sop[:, 1])
dev_alt = compute_interval_in_cents(traj_median_alt[:, 1], traj_MIDI_alt[:, 1])
dev_ten = compute_interval_in_cents(traj_median_ten[:, 1], traj_MIDI_ten[:, 1])
dev_bas = compute_interval_in_cents(traj_median_bas[:, 1], traj_MIDI_bas[:, 1])

print(dev_sop.reshape(-1,1).shape)

In [ ]:
# visualize mean
dev_mean = np.nanmean(np.hstack((dev_sop.reshape(-1,1), dev_alt.reshape(-1,1), dev_ten.reshape(-1,1), dev_bas.reshape(-1,1))), axis=1)
print(dev_mean)

In [ ]:
plt.figure()
plt.plot(sop[:, 0], dev_mean)

In [ ]:
# visualize individual voices
#%matplotlib notebook

fig = plt.figure()
plt.axhline(y=0, color='k')
#plt.plot(sop_mean[:, 0], mean_dev, color='k', marker='.', linestyle='')
plt.plot(sop[:, 0], dev_sop, marker='.', linestyle='')
plt.plot(alt[:, 0], dev_alt, marker='.', linestyle='')
plt.plot(ten[:, 0], dev_ten, marker='.', linestyle='')
plt.plot(bas[:, 0], dev_bas, marker='.', linestyle='')
#plt.ylim((-500, 200))
plt.legend(('', 'Soprano', 'Alto', 'Tenor', 'Bass'))

In [ ]:
import pywebaudioplayer as pwa
import IPython.display
import os

# temporary paths
temp_audio_path = './player.wav'
temp_figure_path = './player.png'

# delete existing files
if os.path.isfile(temp_audio_path):
    os.remove(temp_audio_path)
if os.path.isfile(temp_figure_path):
    os.remove(temp_figure_path)

x, fs = librosa.load("../../../../Google Drive/Dagstuhl Dataset/Audio/norm-downsampled/DLI_QuartetB_Take4_Stereo_STM.wav")
ts2 = pwa.trackswitch([{'title': 'Audio', 'samples': (x, fs), 'path': temp_audio_path}], seekable_image=(fig, temp_figure_path), repeat=True)
IPython.display.HTML(ts2)

In [ ]:
from IPython.display import HTML

HTML(filename="../player.html")